# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as Dfs
import Distributions as Dsts
import CairoMakie as Cmk

In [ ]:
include("./pmf.jl")
import .ProbabilityMassFunction as Pmf

## Odds

Odds [odds = $\frac{P(success)}{P(failure)} = \frac{P(success)}{1 -
P(success)}$], are used in the form of so called **Bayes's Rule**.

Bayes’s Rule is convenient if you want to do a Bayesian update on paper or in
your head. It also sheds light on the important idea of evidence and how we can
quantify the strength of evidence.

In [ ]:
function getOdds(prob::Float64)::Float64
    @assert (0 <= prob <= 1) "prob takes value in range [0-1]"
    return prob / (1 - prob)
end

In [ ]:
# if prob of winning is 75% then the odds are
# 3 to 1
getOdds(0.75)

In [ ]:
# odds in favor (1 to 9)
getOdds(0.1)

In [ ]:
# we may prefer to report odds agains 9:1
getOdds(1-0.1)

We can also do the opposite transformation, below my (BL) convesion for practice

$odds = \frac{p}{1-p}$

$\frac{p}{1-p} = odds$

$\frac{p}{1-p} = odds$ # multiply both sides by (1-p)

$p = odds * (1-p)$ # act on right side: multiply odds by (1-p)

$p = odds - odds * p$ # move (odds*p) to the left with the opposite sign

$p + (odds * p) = odds$ # rewrite left side to different equivalent form

$p * (1 + odds) = odds$ # move (1+odds) to the right with the opposite sign

$p = odds / (1 + odds)$

Below, a Julia's function for that calculation:

In [ ]:
function getProb(odds::T)::T where T<:Union{Rational, Float64}
    return odds / (odds + 1) 
end

In [ ]:
getProb(3/2),
getProb(3//2)

In [ ]:
function getProb(yes::Int, no::Int)::Float64
    return yes / (yes + no) 
end

In [ ]:
getProb(3, 2)

[...] some computations are easier when we work with odds, [...], and some
computations are even easier with log odds, [...].

## Bayes's Rule

So far we have worked with Bayes’s theorem in the “probability form”:

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

Writing $odds(A)$ for odds in favor of `A`, we can express Bayes’s Theorem in “odds form”:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$

In general Bayes's Rule says that the posterior odds are prior odds times the likelihood ratio.


**My comment**

Not sure how this odds form came to be all of a sudden?!

$P(D|B)$ suggests that probability of failure was introduced, since
in the case of two exhaustive mutually exclusive events (success and failure we got):

$P(success) + P(failure) = 1$ or $P(A) + P(B) = 1$

therefore

$P(success) = 1 - P(failure)$ or $P(A) = 1 - P(B)$

So if:

$odds(A) = \frac{P(A)}{1 - P(A)} = \frac{P(A)}{P(B)}$, then

$odds(A|D) = \frac{P(A|D)}{1 - P(A|D)} = \frac{P(A|D)}{P(B|D)}$

Moreover, at the onset the Bayes's theorem had to be written in the $P(A|D)$ and not $P(H|D)$ form.

So:

$P(A|D) = \frac{P(A)P(D|A)}{P(D)}$, instead of

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

Given the above and based on the definition of `getProb(odds)` above we see that:

$P(A) = \frac{odds(A)}{1 - odds{A}} = \frac{odds(A)}{odds{B}}$, therefore we can rewrite

$P(A|D) = \frac{P(A)P(D|A)}{P(D)}$ as

$\frac{odds(A|D)}{odds(B|D)} = \frac{P(A)P(D|A)}{P(D)}$, so i we multiply both sides by $odds(B|D)$ we get

$odds(A|D) = odds(B|D) * \frac{P(A)P(D|A)}{P(D)}$, but I am not sure what happens next.

Anyway, let's get back to the cookie problem:

> Suppose there are two bowls of cookies. Bowl 1 contains 30 vanilla cookies and
> 10 chocolate cookies. Bowl 2 contains 20 of each. Now suppose you choose one of
> the bowls at random and, without looking, select a cookie at random. The cookie
> is vanilla. What is the probability that it came from Bowl 1?

Let's use the **Bayes's Rule**:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$

In [ ]:
# odds(A) = odds(bowl1) = p(bowl1)/p(bowl2)
cookiePriorOdds = 1 # 0.5/0.5
# p(D|A) = p(vanilla|bowl1) = 30/40 = 3/4 = 0.75
# p(D|B) = p(vanilla|bowl2) = 20/40 = 2/4 = 0.5
# likelihood ratio = p(D|A)/p(D|B)
cookieLikelihoodRatio = 0.75 / 0.5
cookiePostOdds = cookiePriorOdds * cookieLikelihoodRatio

In [ ]:
cookiePostProb = getProb(cookiePostOdds)

If we draw another cookie and it's chocolate, we can do another update:

In [ ]:
cookieLikelihoodRatio = (10/40) / (20/40)
cookiePostOdds *= cookieLikelihoodRatio

In [ ]:
cookiePostProb = getProb(cookiePostOdds)

## Oliver's Blood

Problem from MacKay's [Information Theory, Inference, and Learning
Algorithms:](https://www.inference.org.uk/mackay/itila/)

> Two people have left traces of their own blood at the scene of a crime. A
> suspect, Oliver, is tested and found to have type ‘O’ blood. The blood groups of
> the two traces are found to be of type ‘O’ (a common type in the local
> population, having frequency 60%) and of type ‘AB’ (a rare type, with frequency
> 1%). Do these data [the traces found at the scene] give evidence in favor of the
> proposition that Oliver was one of the people [who left blood at the scene]?

What it means evidence in favor of hypothesis. The Bayes's rule says:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$

dividing throuigh by $odds(A)$, we get:

$\frac{odds(A|D)}{odds(A)} = \frac{P(D|A)}{P(D|B)}$

The term on the left is the ratio of the posterior and prior odds. The term on
the right is the likelihood ratio, also called the **Bayes factor**.

If the Bayes factor is greater than 1, that means that the data were more likely
under `A` than under `B`. And that means that the odds are greater, in light of
the data, than they were before.

If the Bayes factor is less than 1, that means the data were less likely under 
`A` than under `B`, so the odds in favor of `A` go down.

Finally, if the Bayes factor is exactly 1, the data are equally likely under
either hypothesis, so the odds do not change.

If Oliver is one of the people who left blood at the crime scene, he accounts
for the ‘O’ sample; in that case, the probability of the data is the
probability that a random member of the population has type ‘AB’ blood, which
is 1%.

If Oliver did not leave blood at the scene, we have two samples to account for.
If we choose two random people from the population, what is the chance of
finding one with type ‘O’ and one with type ‘AB’? Well, there are two ways it
might happen:
- 0 and AB (prob 0.6 * 0.01 = 0.006)
- AB and 0 (prob 0.01 * 0.6 = 0.006)

So, prob = 0.012 in total

In [ ]:
# p(D|A) = p(gr0 is Oliver and other guy acccounts for AB)
like1 = 0.01
# p(D|B) = p(gr0 and grAB are left by someone else)
like2 = 0.6*0.01*2
likelihoodRatio = like1/like2

So little evidence against the hypothesis

In [ ]:
getProb(likelihoodRatio) # prob oliver is guilty

If this result still bothers you, this way of thinking might help: the data
consist of a common event, type ‘O’ blood, and a rare event, type ‘AB’ blood. If
Oliver accounts for the common event, that leaves the rare event unexplained. If
Oliver doesn’t account for the ‘O’ blood, we have two chances to find someone in
the population with ‘AB’ blood. And that factor of two makes the difference.

### Exercise 0.1

Suppose that based on other evidence, you prior belief in Oliver’s guilt is 90%.
How much would the blood evidence in this section change your beliefs? What if
you initially thought there was only a 10% chance of his guilt?



Bayes's Rule.

$odds(A|D) = odds(A) \frac{P(D|A)}{P(D|B)}$

So, previously prior odds [odds(A) were 1, we assumed it equally likely that
Oliver accounted for the blood sample O (p = 0.5) as that he did not (p = 0.5)].
And now the probablity is 0.9 (90%), so the prior odds [odds(A)] are
0.9/0.1 = 9

In [ ]:
# prior probability of Oliver's guild 90% = 0.9
postOdds = getOdds(0.9) * like1/like2

In [ ]:
# prior probability of Oliver's guild 90% = 0.9
getProb(postOdds)

Bayes's Rule.

$odds(A|D) = odds(A) \frac{P(D|A)}{P(D|B)}$

So, previously prior odds [odds(A) were 1, we assumed it equally likely that
Oliver accounted for the blood sample O (p = 0.5) as that he did not (p = 0.5)].
And now the probablity is 0.1 (10%), so the prior odds [odds(A)] are
0.1/0.9 = 1/9 = 0.11(1)

In [ ]:
# prior probability of Oliver's guild 10% = 0.1
postOdds = getOdds(0.1) * like1/like2

In [ ]:
# prior probability of Oliver's guild 10% = 0.1
getProb(postOdds)

## Addends

The second half of this chapter is about distributions of sums and results of
other operations.

Let's start with simple 6-sided dice.

In [ ]:
# creates a Pmf with names - number of dots on a dice
# priors - probability of obtaining the number of dots on a dice
function makeDice(nSides::Int = 6)::Pmf.Pmf{Int}
    return Pmf.getPmfFromSeq(1:nSides |> collect)
end

In [ ]:
dice = makeDice(6)

In [ ]:
fig = Cmk.Figure()
Cmk.barplot(fig[1, 1], dice.names, dice.priors, color = "lightblue",
    axis=(;title="Prior distribution of dots after a dice throw",
    xlabel="Outcome (number of dots)",
    ylabel="PMF",
    xticks=1:6)
)
fig

In [ ]:
function sumProbsByNames(names::Vector{Int}, probs::Vector{Float64})::Dict{Int,Float64}
    @assert length(names) == length(probs)
    res::Dict{Int,Float64} = Dict()
    for i in eachindex(names)
        res[names[i]] = get(res, names[i], 0) + probs[i]
    end
    return res
end

In [ ]:
"""
Inspired by a similar function found in empiricaldist by Allen Downey
    applies a fn to cartesianProduct(pmf1.names, pmf2.names)
    applies * to cartesianProduct(pmf1.prior, pmf2.priors), so P(A) and P(B)

args:
fn - function accepting two Int64s as input and returning Int64 as output
"""
function convolveDist(pmf1::Pmf.Pmf{Int}, pmf2::Pmf.Pmf{Int}, fn::Function)::Pmf.Pmf{Int}
    # Iterators.product(vec1, vec2) gives cartesian product of two vects [(v1.0, v2.0), (v1.0, v2.1), etc.]
    newNames::Vector{Int} = [[fn(a, b) for (a, b) in Iterators.product(pmf1.names, pmf2.names)]...]
    newPriors::Vector{Float64} = [[a * b for (a, b) in Iterators.product(pmf1.priors, pmf2.priors)]...] 
    probs::Dict{Int, Float64} = sumProbsByNames(newNames, newPriors)
    orderedKeys::Vector{Int64} = sort(collect(keys(probs)))
    orderedVals::Vector{Float64} = [probs[k] for k in orderedKeys]
    return Pmf.Pmf(orderedKeys, orderedVals ./ sum(orderedVals))
end

In [ ]:
function addDist(pmf1::Pmf.Pmf{Int}, x::Int)::Pmf.Pmf{Int}
    return Pmf.Pmf(pmf1.names .+ x, pmf1.priors)
end

In [ ]:
function addDist(pmf1::Pmf.Pmf{Int}, pmf2::Pmf.Pmf{Int})::Pmf.Pmf{Int}
    return convolveDist(pmf1, pmf2, +)
end

In [ ]:
twice = addDist(dice, dice)

In [ ]:
fig = Cmk.Figure()
Cmk.barplot(fig[1, 1], twice.names, twice.priors, color = "wheat",
    axis=(;title="Prior distribution of sum of dots after a two dice throw",
    xlabel="Outcome (sum of dots)",
    ylabel="PMF",
    xticks=1:12)
)
fig

In [ ]:
"""
Add all pmfs in a sequence 
"""
function addDistSeq(seq::Vector{Pmf.Pmf{Int}})::Pmf.Pmf{Int}
    return reduce(addDist, seq) 
end

In [ ]:
thrice = addDistSeq(repeat([dice], 3))

In [ ]:
fig = Cmk.Figure()
ax1, l1 = Cmk.lines(fig[1, 1], dice.names, dice.priors,
    color="blue", linestyle=:solid, linewidth=2,
    axis=(; title="Distribution of sums",
    xlabel="Outcome (sum of dots)",
    ylabel="PMF",
    xticks=1:18)
)
l2 = Cmk.lines!(fig[1, 1], twice.names, twice.priors,
    color = "orange", linestyle=:dash, linewidth=2)
l3 = Cmk.lines!(fig[1, 1], thrice.names, thrice.priors,
    color="green", linestyle=:dot, linewidth=2)
Cmk.axislegend(ax1,
    [l1, l2, l3],
    ["one dice", "two dice", "three dice"],
    "# dice used"
)
fig

## Gluten Sensitivity

A [paper](https://onlinelibrary.wiley.com/doi/full/10.1111/apt.13372) on gluten
sensitivity.

Out of 35 subjects, 12 correctly identified the gluten flour based on resumption
of symptoms while they were eating it.  Another 17 wrongly identified the
gluten-free flour based on their symptoms, and 6 were unable to distinguish.

So here’s the question: based on this data, how many of the subjects are
sensitive to gluten and how many are guessing?

Solution using Bayes’s Theorem. Assumptions:
- People who are sensitive to gluten have a 95% chance of correctly identifying
gluten flour under the challenge conditions, and
- People who are not sensitive have a 40% chance of identifying the gluten flour
by chance (and a 60% chance of either choosing the other flour or failing to
distinguish).

These values are arbitrary, but reasonable.

First, assume that we know how many subjects are sensitive.
Second, using the likelihood of the data, compute the posterior distribution of
the number of sensitive patients.

The first is the **forward problem**; the second is the **inverse problem**.

### The Forward Problem

Suppose we know that 10 of the 35 subjects are sensitive to gluten.

In [ ]:
n = 35
nSensitive = 10
nInsenstive = n - nSensitive

In [ ]:
dstSensitive = Pmf.getBinomialPmf(nSensitive, 0.95)
dstInsensitive = Pmf.getBinomialPmf(nInsenstive, 0.4)

In [ ]:
dstTotal = addDist(dstSensitive, dstInsensitive)

In [ ]:
fig = Cmk.Figure()
ax1, l1 = Cmk.lines(fig[1, 1],
    dstSensitive.names, dstSensitive.priors,
    color="blue", linestyle=:dot, linewidth=2,
    axis=(;title="Gluten sensitivity",
        xlabel="Number of correct identifications",
        ylabel="PMF",
        xticks=0:5:35)
)
l2 = Cmk.lines!(fig[1, 1],
    dstInsensitive.names, dstInsensitive.priors,
    color="orange", linestyle=:dash, linewidth=2,
)
l3 = Cmk.lines!(fig[1, 1],
    dstTotal.names, dstTotal.priors,
    color="green", linestyle=:solid, linewidth=2,
)
Cmk.axislegend(ax1,
    [l1, l2, l3],
    ["sensitive", "insensitive", "total"],
    "distribution"
)
fig

We expect most of the sensitive subjects to identify the gluten flour correctly.
Of the 25 insensitive subjects, we expect about 10 to identify the gluten flour
by chance. So we expect about 20 correct identifications in total.

This is the answer to the forward problem: given the number of sensitive
subjects, we can compute the distribution of the data.

### The Inverse Problem

Now let’s solve the inverse problem: given the data, we’ll compute the posterior
distribution of the number of sensitive subjects.

In [ ]:
tableSensitives = Dict{Int, Pmf.Pmf}()

for nSensitive in 0:n
    nInsensitive = n - nSensitive
    dstSensitive = Pmf.getBinomialPmf(nSensitive, 0.95)
    dstInsensitive = Pmf.getBinomialPmf(nInsenstive, 0.4)
    dstTotal = addDist(dstSensitive, dstInsensitive)
    tableSensitives[nSensitive] = dstTotal
end;

In [ ]:
fig = Cmk.Figure()
ax1, l1 = Cmk.lines(fig[1, 1],
    tableSensitives[0].names, tableSensitives[0].priors,
    color="blue", linestyle=:solid, linewidth=2,
    axis=(;title="Gluten sensitivity",
        xlabel="Number of correct identifications",
        ylabel="PMF",
        xticks=0:5:35)
)
Cmk.ylims!(ax1, 0, 0.25)
l2 = Cmk.lines!(fig[1, 1],
    tableSensitives[10].names, tableSensitives[10].priors,
    color="orange", linestyle=:solid, linewidth=2,
)
l3 = Cmk.lines!(fig[1, 1],
    tableSensitives[20].names, tableSensitives[20].priors,
    color="green", linestyle=:dash, linewidth=2,
)
l4 = Cmk.lines!(fig[1, 1],
    tableSensitives[30].names, tableSensitives[30].priors,
    color="red", linestyle=:dot, linewidth=2,
)
Cmk.axislegend(ax1,
    [l1, l2, l3, l4],
    ["0/35", "10/35", "20/35", "30/35"],
    "num sensitive / total participants"
)
fig

In [ ]:
# in task we got 12 positive identifications (out of 0:35)
glutenLikelihood12 = [Pmf.getPriorByName(tableSensitives[i], 12) for i in 0:35];

In [ ]:
gluten12 = Pmf.getPmfFromSeq(0:35 |> collect);
gluten12.likelihoods = gluten12.priors .* glutenLikelihood12;
Pmf.updatePosteriors!(gluten12, true);

In [ ]:
# for comparison, 20 correct identifications
glutenLikelihood20 = [Pmf.getPriorByName(tableSensitives[i], 20) for i in 0:35];

In [ ]:
gluten20 = Pmf.getPmfFromSeq(0:35 |> collect);
gluten20.likelihoods = gluten20.priors .* glutenLikelihood20;
Pmf.updatePosteriors!(gluten20, true);

In [ ]:
fig = Cmk.Figure()
ax1, l1 = Cmk.lines(fig[1, 1],
    gluten12.names, gluten12.posteriors,
    color="purple", linestyle=:solid, linewidth=2,
    axis=(;title="Gluten sensitivity (posterior distributions)",
        xlabel="Number of sensitive subjects",
        ylabel="PMF",
        xticks=0:5:35)
)
Cmk.ylims!(ax1, 0, 0.25)
l2 = Cmk.lines!(fig[1, 1],
    gluten20.names, gluten20.posteriors,
    color="orange", linestyle=:solid, linewidth=2,
)
Cmk.axislegend(ax1,
    [l1, l2],
    ["# 12", "# 20"],
    "correct identifications"
)
fig

In [ ]:
Pmf.getNameMaxPosterior(gluten12)

In [ ]:
Pmf.getNameMaxPosterior(gluten20)

## Summary

Bayes's Rule is a way to quantify the strength of evidence using a likelihood
ratio or Bayes factor.

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$

The function `add_dist`, computes the distribution of a sum of two
distributions.

We use it for forward problems:
- given the parameters of a system, we can compute the distribution of the data

We use it for inverse problems:
- given the data, we can compute the distribution of the parameters

## Exercises

### Exercise 1

Use Bayes's Rule to solve the Elvis problem from Ch03 (Distributions)

Elvis Presley had a twin brother who died at birth. What is the probability that
Elvis was an identical twin?

In 1935, about 2/3 of twins were fraternal and 1/3 were identical. The question
contains two pieces of information we can use to update this prior.

In [ ]:
# odds(A|D) - identical twin given twin brother
# odds(A) - identical twin
ex1OddsA = getOdds(1/3)
# p(D|A) - twin brother given identical twin
ex1PDgivenA = 1
# p(D|B) - twin brother given fraternal twin
ex1PDgivenB = 1/2

In [ ]:
# odds(A|D) = odds(A) * (P(D|A)/P(D|B))
oddsAgivenD = ex1OddsA * (ex1PDgivenA / ex1PDgivenB)

In [ ]:
getProb(oddsAgivenD) # 0.49999... = ~0.5 like in the solution to ch03/ex4
# not sure why the solution from Allen Downey for this exercise differs (p = 0.555...)

### Exercise 2

The following is an interview question that appeared on
[glassdoor.com](https://www.glassdoor.com/Interview/You-re-about-to-get-on-a-plane-to-Seattle-You-want-to-know-if-you-should-bring-an-umbrella-You-call-3-random-friends-of-y-QTN_519262.htm),
attributed to Facebook:

> You’re about to get on a plane to Seattle. You want to know if you should
> bring an umbrella. You call 3 random friends of yours who live there and ask
> each independently if it’s raining. Each of your friends has a 2/3 chance of
> telling you the truth and a 1/3 chance of messing with you by lying. All 3
> friends tell you that “Yes” it is raining. What is the probability that it’s
> actually raining in Seattle?

Use Bayes’s Rule to solve this problem. As a prior you can assume that it rains
in Seattle about 10% of the time.

I (BL) will assume that the '2/3 chance' and '1/3 chance' in the description above actually means probability.

In [ ]:
# odds(A|D) = odds(A) * (P(D|A)/P(D|B))

# odds(A|D) - raining given three Yes

# oddds(A) - raining
ex2OddsA = getOdds(0.1)

# P(D|A) - three yes, given it is raining
ex2PDgivenA = (2/3)^3 # 2//3 * 2//3 * 2//3 (and in probs is *)

# P(D|B) - three yes, given it is not raining
ex2PDgivenB = (1/3)^3 # 1//3 * 1//3 * 1//3 (and in probs is *)

ex2OddsAGivenD = ex2OddsA * (ex2PDgivenA / ex2PDgivenB)

In [ ]:
getProb(ex2OddsAGivenD)

### Exercise 3

According to the
[CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/health_effects/effects_cig_smoking/),
people who smoke are about 25 times more likely to develop lung cancer than
nonsmokers.

Also according to the
[CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm),
about 14% of adults in the U.S. are smokers. If you learn that someone has lung
cancer, what is the probability they are a smoker?

I (BL) will assume that this someone is from the U.S.

In [ ]:
# odds(A|D) = odds(A) * (P(D|A)/P(D|B))

# odds(A|D) - smoker given lung cancer present

# odds(A) - smoker
ex3OddsA = getOdds(0.14)

# P(D|A) - lung cancer given smoker
# P(D|B) - lung cancer given non-smoker
# likelihood - P(D|A)/P(D|B)
ex3PDgivenAbyB = 25

ex3OddsAGivenD = ex3OddsA * ex3PDgivenAbyB

In [ ]:
getProb(ex3OddsAGivenD)

### Exercise 4

In Dungeons & Dragons, the amount of damage a goblin can withstand is the sum of
two six-sided dice. The amount of damage you inflict with a short sword is
determined by rolling one six-sided die. A goblin is defeated if the total
damage you inflict is greater than or equal to the amount it can withstand.

Suppose you are fighting a goblin and you have already inflicted 3 points of
damage. What is your probability of defeating the goblin with your next
successful attack?

Hint: You can use `Pmf.sub_dist` to subtract a constant amount, like 3, from a
`Pmf`.

In [ ]:
function subtractDist(pmf1::Pmf.Pmf{Int}, x::Int)::Pmf.Pmf{Int}
    return Pmf.Pmf(pmf1.names .- x, pmf1.priors)
end

In [ ]:
function subtractDist(pmf1::Pmf.Pmf{Int}, pmf2::Pmf.Pmf{Int})::Pmf.Pmf{Int}
    return convolveDist(pmf1, pmf2, -)
end

In [ ]:
ex4Dice = Pmf.getPmfFromSeq(1:6 |> collect)

In [ ]:
# goblin, full HP
goblinHpBeforeRound1 = addDist(ex4Dice, ex4Dice)

In [ ]:
# goblin with 3 pts of damage already inflicted
goblinHpAfterRound1 = subtractDist(goblinHpBeforeRound1, 3)
# goblin is still alive, so >= 1 hp 
goblinIndsAliveAfterRound1 = findall((>)(0), goblinHpAfterRound1.names)
# or findall(x -> x > 0, goblinHpAfterRound1.names)
goblinHpBeforeRound2 = Pmf.Pmf(
    goblinHpAfterRound1.names[goblinIndsAliveAfterRound1],
    goblinHpAfterRound1.priors[goblinIndsAliveAfterRound1]
    )
# normalize priors
goblinHpBeforeRound2.priors ./= sum(goblinHpBeforeRound2.priors)
goblinHpBeforeRound2

In [ ]:
goblinHpAfterRound2 = subtractDist(goblinHpBeforeRound2, ex4Dice)

In [ ]:
goblinIndsDeadAfterRound2 = findall((<=)(0), goblinHpAfterRound2.names)
# or: findall(x -> x <= 0, goblinHpAfterRound2.names)

In [ ]:
fig = Cmk.Figure()
ax1, b1 = Cmk.barplot(fig[1, 1],
            goblinHpAfterRound2.names, goblinHpAfterRound2.priors,
            color = "lightblue",
            axis=(;title="D&D goblin HP points after 2 hits",
            xlabel="HP points left",
            ylabel="PMF",
            xticks=-5:8)
)
# overlying 'dead' goblin bars over previously drawn bars
b2 = Cmk.barplot!(fig[1, 1],
            goblinHpAfterRound2.names[goblinIndsDeadAfterRound2],
            goblinHpAfterRound2.priors[goblinIndsDeadAfterRound2],
            color = "lightgray",
)
Cmk.axislegend(ax1,
    [b1, b2],
    ["alive", "dead"],
    "goblin's status"
)
fig

In [ ]:
probGoblinDeadAfter2Rounds = goblinHpAfterRound2.priors[goblinIndsDeadAfterRound2] |> sum

## Exercise 5

Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die. I
choose one of the dice at random, roll it twice, multiply the outcomes, and
report that the product is 12. What is the probability that I chose the 8-sided
die?

Hint: `Pmf` provides a function called `mul_dist` that takes two `Pmf` objects
and returns a Pmf that represents the distribution of the product.

In [ ]:
function multDist(pmf1::Pmf.Pmf{Int}, x::Int)::Pmf.Pmf{Int}
    return Pmf.Pmf(pmf1.names .* x, pmf1.priors)
end

In [ ]:
function multDist(pmf1::Pmf.Pmf{Int}, pmf2::Pmf.Pmf{Int})::Pmf.Pmf{Int}
    return convolveDist(pmf1, pmf2, *)
end

In [ ]:
ex5diceTypes = [6, 8, 12]

In [ ]:
ex5priors = Dict(d => 1/3 for d in ex5diceTypes)

In [ ]:
ex5dicesPmfs = Dict(
    d => Pmf.getPmfFromSeq(1:d |> collect) for d in ex5diceTypes)

In [ ]:
ex5dicesPmfs2throws = Dict(
    d => multDist(ex5dicesPmfs[d], ex5dicesPmfs[d]) for d in ex5diceTypes)

In [ ]:
ex5likelihoods = Dict(
    d => Pmf.getPriorByName(ex5dicesPmfs2throws[d], 12) for d in ex5diceTypes)

In [ ]:
# classic Bayes's Theorem
# P(H|D) = [P(H) * P(D|H)] / P(D)
# P(D) - normalizing constant, that we omit, or define as sum of posteriors
ex5posteriors = Dict(
    d => ex5priors[d] * ex5likelihoods[d] for d in ex5diceTypes)

In [ ]:
# normalizing posteriors
ex5pD = ex5posteriors |> values |> sum
ex5posteriors = Dict(
    d => ex5posteriors[d] / ex5pD for d in ex5diceTypes)